This code is for the Gottwald Model, a simplified climate model, combining a Lorenz-84 chaotic atmosphere and a Stommel simplified ocean 'box' model.

The Lorenz-84 atmosphere involves the variables:
    - x, the east-west strength of westerlies
    - y and z, north-south circulation of eddies (y^2 + z^2 = total eddy energy)

The Stommel model is a simplified ocean model without ice, and 2 main variables:
    - T: average temperature
    - S: average salinity
    
This model divides the ocean into 2 boxes, a northern latitude and a southern latitude, finally giving 4 variables-the 2 main variables of each box: (T1, S1) and (T2, S2). The idea with the Stommel model is to produce a circulation strength by finding the density difference between the two boxes, resulting in only two variables: T1-T2, and S1-S2.

The Gottwald model reproduced in this code couples the Stommel and Lorenz-84 models to give a fast, chaotic atmosphere interacting with a slow ocean. Temperature changes in the atmosphere continuously perturb the ocean, while fluxes in the ocean slowly shift the mean state of the atmosphere. The variable epsilon_f controls the time dynamics of the atmosphere coupled with the ocean. As epsilon_f approaches infinity, time dynamics of the atmosphere approach infinity, and the effect on the ocean becomes infinitesimally small, essentially representing noise.


This code has three options for running the model:
1. Regular model (as described above)
2. Scaled model (for use in edge tracking)
3. Seasonal model (for looking at seasonal changes)

I am working on changing parameter values for the model, to see if strengthening the coupling term between ocean and atmosphere will produce any bifurcations. I have therefore added a loop to run the model with different values of epsilon_f, and save the output as a csv file.


In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

In [29]:
def smoothabs(x, xi=10000):
    """Smooth absolute value function"""
    return x * np.tanh(x * xi)


In [30]:
 def gottwald_noice(t, u, p, stochsys=True):
    """Gottwald model functions without sea ice"""
    x, y, z, T, S = u
    if stochsys:
        pvec = p[0] if isinstance(p, list) else p
    else:
        pvec = p
    
    (a, b, mu, epsilon_a, epsilon_f, F0, F1, G0, G1, 
     theta_0, theta_1, sigma_0, sigma_1, x_mean, Delta_mean) = pvec
    
    Delta = y**2 + z**2
    T_surf = theta_0 + theta_1 * (x - x_mean)/(np.sqrt(epsilon_f))
    S_surf = sigma_0 + sigma_1 * (Delta - Delta_mean)/(np.sqrt(epsilon_f))
    
    dx = 1/epsilon_f*(-Delta - a*(x - F0 - F1*T))
    dy = 1/epsilon_f*(x*y - b*x*z - (y - G0 + G1*T))
    dz = 1/epsilon_f*(b*x*y + x*z - z)
    dT = -1/epsilon_a*(T - T_surf) - T - mu*smoothabs(S-T)*T
    dS = S_surf - S - mu*smoothabs(S-T)*S
    
    return np.array([dx, dy, dz, dT, dS])

In [31]:
#these functions could be integrated, only one difference here
def gottwald_noice_scaled(t, u, p, stochsys=True):
    """Rescaled (x/y/z) for edge tracking"""
    x, y, z, T, S = u
    if stochsys:
        pvec = p[0] if isinstance(p, list) else p
    else:
        pvec = p
    
    (a, b, mu, epsilon_a, epsilon_f, F0, F1, G0, G1, 
     theta_0, theta_1, sigma_0, sigma_1, x_mean, Delta_mean, s) = pvec
    
    Delta = y**2 + z**2
    T_surf = theta_0 + theta_1 * (s*x - x_mean)/(np.sqrt(epsilon_f))
    S_surf = sigma_0 + sigma_1 * (s**2*Delta - Delta_mean)/(np.sqrt(epsilon_f))

    #for integration, could make s = abs(1)
    dx = 1/epsilon_f*(-s*Delta - a/s*(s*x - F0 - F1*T))
    dy = 1/epsilon_f*(s*x*y - s*b*x*z - (s*y - G0 + G1*T)/s)
    dz = 1/epsilon_f*(s*b*x*y + s*x*z - z)
    dT = -1/epsilon_a*(T - T_surf) - T - mu*smoothabs(S-T)*T
    dS = S_surf - S - mu*smoothabs(S-T)*S
    
    return np.array([dx, dy, dz, dT, dS])

In [32]:
def gottwald_noice_seasons(t, u, p):
    """Gottwald model functions without sea ice, but with seasonal cycle"""
    x, y, z, T, S = u
    (a, b, mu, epsilon_a, epsilon_f, F0, F1, G0, G1, 
     theta_0, theta_1, sigma_0, sigma_1, x_mean, Delta_mean, td, F2, G2) = p[0]
    
    omega = 2.0 * np.pi * td  # annual frequency
    Delta = y**2 + z**2
    T_surf = theta_0 + theta_1 * (x - x_mean)/(np.sqrt(epsilon_f))
    S_surf = sigma_0 + sigma_1 * (Delta - Delta_mean)/(np.sqrt(epsilon_f))
    
    dx = 1/epsilon_f*(-Delta - a*(x - F0 - F1*T - F2*np.cos(omega * t)))
    dy = 1/epsilon_f*(x*y - b*x*z - (y - G0 + G1*T - G2*np.cos(omega * t)))
    dz = 1/epsilon_f*(b*x*y + x*z - z)
    dT = -1/epsilon_a*(T - T_surf) - T - mu*smoothabs(S-T)*T
    dS = S_surf - S - mu*smoothabs(S-T)*S
    
    return np.array([dx, dy, dz, dT, dS])

In [72]:
def param_gwn_default(f,param_l84=None, sigma_0=0.926): #**kwargs):
    """return parameter values"""
    if param_l84 is not None:
        a, b, F0, G0 = param_l84
    else:
        param_l84 = [0.25, 4., 8., 1.]  # default L84 params [a, b, F0, G0]
        
    x_std = 0.513
    Delta_std = 1.071
    x_mean = 1.0147
    Delta_mean = 1.7463  # long-run results from default L84
    
    # Unpack L84 params
    a, b, F0, G0 = param_l84
    
    # Other default model params
    mu = 7.5
    theta_0 = 1.
    # sigma_0 = 0.9  # Stommel params (is varied) 0.926, 0.932
    epsilon_a = f
#change to f, add as parameter
    epsilon_f = 0.009 # 0.0083 # timescales
    F1 = 0.1
    G1 = 0.  # coupling params
    perturb_scaling = 0.01  # coupling strength of L84 to Stommel
    
    # Coupling params from L84 run/default
    theta_1 = min(theta_0, perturb_scaling/x_std)
    sigma_1 = min(sigma_0, perturb_scaling/Delta_std)
   
    
    return [a, b, mu, epsilon_a, epsilon_f, F0, F1, G0, G1, 
            theta_0, theta_1, sigma_0, sigma_1, x_mean, Delta_mean]

In [105]:
#reduced tolerance
def simulate_gottwald_noice(initial_conditions, params, t_span, t_eval=None, stochsys=True):
    """Simulate the Gottwald model without sea ice"""
    sol = solve_ivp(
        lambda t, y: gottwald_noice(t, y, params, stochsys),
        t_span,
        initial_conditions,
        t_eval=t_eval,
        method='Radau',
        rtol=1e-3,
        atol=1e-6
    )
    return sol

In [102]:
# Example usage
if __name__ == "__main__":
    matplotlib.use('Agg')
    
    # Get default Gottwald parameters
    params_gwn = param_gwn_default()
    print(f"Default Gottwald parameters:")
    print(params_gwn)
    
    # Simulate Gottwald model
    initial_gwn = [1.0, 0.5, 0.5, 1.0, 0.9]  # [x, y, z, T, S]
    # params_gwn_wrapped = [params_gwn]  # Wrap in list for stochsys format
    tmax = 1000 # 00
    t_eval = np.linspace(0, tmax, tmax* 2)

    sol_gwn = simulate_gottwald_noice(initial_gwn, params_gwn, (0, tmax), 
                                       t_eval=t_eval, stochsys=False)

'''
    fig, ax = plt.subplots(1, 3, figsize=(4 * 2.5, 3))
    ax[0].plot(sol_gwn.t, sol_gwn.y[3], label="T")
    ax[0].plot(sol_gwn.t, sol_gwn.y[4], label="S")
    ax[0].legend()
    ax[0].set_xlabel("t")
    ax[1].plot(sol_gwn.y[0][:1000], sol_gwn.y[1][:1000])
    ax[1].plot(sol_gwn.y[0][-1000:], sol_gwn.y[1][-1000:])
    ax[1].set_xlabel("x")
    ax[1].set_ylabel("y")
    ax[2].plot(sol_gwn.y[1][:1000], sol_gwn.y[2][:1000])
    ax[2].plot(sol_gwn.y[1][-1000:], sol_gwn.y[2][-1000:])    
    ax[2].set_xlabel("y")
    ax[2].set_ylabel("z")
    fig.tight_layout()   

    fig.savefig('gottwald_noice.png')
'''

TypeError: param_gwn_default() missing 1 required positional argument: 'f'

In [106]:
'''
Run long simulations to observe if there are any random jumps from one attractor to another
Will need to determine which initial conditions change the attractor (so in this case, the only variable changing is epsilon_f)
So will need to save in the output, changes in epsilon_f, but tbh probably want to see how the other variables evolve over time in 
relation to epsilon f
So will save x,y,z,T,S,epsilon_f, t (time)
Right now T is sol_gwn.y[3], S is sol_gwn.y[4], t is sol_gwn.t, x is sol_gwn.y[0], y is sol_gwn.y[1], z is sol_gwn.y[2]

Create function where epsilon_f is varied, approaching zero, run solver for each value of epsilon f
'''
#3e-2,3e-3,3e-4,3e-5,3e-6,

def change_epsilon():

    f_list = [3e-7,3e-8,3e-9,3e-10]
    for f in f_list:
    
        if __name__ == "__main__":
            matplotlib.use('Agg')
            
            # Get default Gottwald parameters
            params_gwn = param_gwn_default(f)
            print(f"Default Gottwald parameters:")
            print(params_gwn)
            
            # Simulate Gottwald model
            initial_gwn = [1.0, 0.5, 0.5, 1.0, 0.9]  # [x, y, z, T, S]
            # params_gwn_wrapped = [params_gwn]  # Wrap in list for stochsys format
            tmax = 1000 # 00
            t_eval = np.linspace(0, tmax, tmax*1)
        
            sol_gwn = simulate_gottwald_noice(initial_gwn, params_gwn, (0, tmax), 
                                               t_eval=t_eval, stochsys=False)
    
            data = {
                't': sol_gwn.t,
                'x': sol_gwn.y[0],
                'y': sol_gwn.y[1],
                'z': sol_gwn.y[2],
                'T': sol_gwn.y[3],
                'S': sol_gwn.y[4]
            }
            
            df = pd.DataFrame(data)
            df.to_csv(f"gottwald_1000_years_{f}.csv")
    
    return 


'''
data = {
    't': sol_gwn.t,
    'x': sol_gwn.y[0],
    'y': sol_gwn.y[1],
    'z': sol_gwn.y[2],
    'T': sol_gwn.y[3],
    'S': sol_gwn.y[4]
}

test = pd.DataFrame(data)
test.head()
test.size
'''

"\ndata = {\n    't': sol_gwn.t,\n    'x': sol_gwn.y[0],\n    'y': sol_gwn.y[1],\n    'z': sol_gwn.y[2],\n    'T': sol_gwn.y[3],\n    'S': sol_gwn.y[4]\n}\n\ntest = pd.DataFrame(data)\ntest.head()\ntest.size\n"

In [107]:
change_epsilon()

Default Gottwald parameters:
[0.25, 4.0, 7.5, 3e-07, 0.009, 8.0, 0.1, 1.0, 0.0, 1.0, 0.01949317738791423, 0.926, 0.009337068160597572, 1.0147, 1.7463]
solver number:  1
Default Gottwald parameters:
[0.25, 4.0, 7.5, 3e-08, 0.009, 8.0, 0.1, 1.0, 0.0, 1.0, 0.01949317738791423, 0.926, 0.009337068160597572, 1.0147, 1.7463]
solver number:  1
Default Gottwald parameters:
[0.25, 4.0, 7.5, 3e-09, 0.009, 8.0, 0.1, 1.0, 0.0, 1.0, 0.01949317738791423, 0.926, 0.009337068160597572, 1.0147, 1.7463]
solver number:  1
Default Gottwald parameters:
[0.25, 4.0, 7.5, 3e-10, 0.009, 8.0, 0.1, 1.0, 0.0, 1.0, 0.01949317738791423, 0.926, 0.009337068160597572, 1.0147, 1.7463]
solver number:  1
